----YAPAY ZEKA VERİ ÖN İŞLEME DERSİ----

![](http://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSDPogmH0eh6Co1on4XoHu3H_TNFl-ZSRyORg&usqp=CAU)

King County, Washington State şehirlerindeki emlak fiyatlarının çeşitli regresyon modelleri ile tahmin edimesi amacı ile paylaşılan bu veri setini kullanarak veri ön işleme yapacağız. 

Kaggle web sitesi notebook açtığımız zaman aşağıdaki kütüpaneleri hazır olarak vermektedir. Biz kendi bilgisayarımızdaki PyCharm veya Spyder gibi programlar ile çalıştığımızda el ile bu kütüphaneleri yükleriz. Numpy kütüphanesi bizim lineer cebir kütüphanesi dediğimiz matris işlemlerini yapmamıza olanak sağlayan bir kütüphanedir. Pandas kütüphanesi ise veri üzerinde çalışmamıza; okuma, filtreleme, satır sütun seçme gibi işlemleri yapmamıza olanak sağlar. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Bu aşamada pandas kütüphanesinin read_csv fonksiyonunu kullanarak veri setimizi bir değişkene atıyoruz.

In [ ]:
dataset = pd.read_csv("../input/kc-house-data/kc_house_data.csv")

Veri setimizi dataset isimli değişkene başarılı bir şekilde atadık. Şimdi veri setimizi incelemek için baştan 5 satırı yazdırıp inceleyelim.

In [ ]:
dataset.head()

Veri setinin kaç satır ve sütundan oluştuğunu incelemek için shape fonksiyonunu kullanıyoruz.

In [ ]:
dataset.shape

Verimiz 21 sütun 21613 satırdan oluşmakta.

In [ ]:
dataset.columns

columns fonksiyonu ile veri setinin sütun isimlerini inceleyebiliriz. Veri setimizi içerisinde tahmin etmemiz gereken sütun price, yani fiyat sütunu. Diğer sütunlara baktığımızda veri setimizin oda sayısı, banyo sayısı, evin büyüklüğü, manzara, deniz kenarında olması, yapım yılı, konumu gibi bigileri içerdiğini görebiliyoruz. 

info() fonksiyonu ile veri seti içerisindeki sütunların tiplerini tespit ediyoruz ve boş, eksik değer olup olmadığına bakıyoruz.

In [ ]:
dataset.info()

Veri setinin içeirsinde boş değer olmadığı açık olduğundan boş değer doldurma ile ilgili bir işlem yapmamıza gerek kalmıyor.

Veri setini incelediğimizde ilk sütunun id olduğunu, ikinci sütunun ise tarih olduğunu görüyoruz. Tahminleme algoritmasında id değerinin bir önemi olmadığını ve bu sütunu veri seti içerisinden çıkarmamızın doğru bir yaklaşım olacağını söylemek yanlış olmaz. Tarih sütununu da uygun biçimde düzenleyemeyeceğimizden yine veri setimizden çıkaracağız. Yine zipcode değerinin de algoritma için bir anlam ifade etmeyeceğini ve çıkarmamız gerektiğini söyleyebiliriz. Bu problem için tahmin edilmesi gereken değerin "price"olduğu açık. Öyle ise artık feature sütunlarımızı ve tahmin edilmesi gereken değerimizi ayırabiliriz.


Veri setimizde 3. sütundan 15. sütuna kadar olan değerleri ayırıyoruz. Sonra da 17. sütundan 21.sütuna kadar olan değerlerle birleştirerek feature sütunlarımızı  oluşturuyoruz. 16. sütunumuz zipcode. Bu sebeple o sütunu dışarıda bıraktık. Sütunları ayırmak için iloc fonksiyonunu kullanacağız. İki parça olan feature'larımızı birleştirmek için ise numpy kütüphanesinden append fonksiyonunu kullanacağız.


In [ ]:
X = dataset.iloc[:, 3:15].values
X = np.append(X, dataset.iloc[:,17:21].values, axis = 1)

iloc fonksiyonunu inceleyecek olursak; köşeli parantez içerisindeki ilk parametre seçmek istediğimiz sayıları, noktalı virgülden sonraki parametreler ise seçmek istediğimiz sütunları temsil etmektedir. Tüm satırları seçmek istiyorsak yukarıdaki gibi noktalı virgülden önce bir şey yazmayız.

Mesela 1'den 5'e kadar olan satırları ve 4'ten 9'a kadar olan sütunları seçmek istesek dataset.iloc[1:5:,4:9] şeklinde yazarız.

Append fonksiyonu ise array tipindeki verilerimizi birleştirmemize yarıyor. append fonksiyonunun içerisine birleştirmek istediğimiz verileri parametre olarak yazarız. axis ise birleştirmenin yatay ya da dikey olarak yapılacağını belirler. Biz burada sütunları birleştireceğimiz için '1' değerini yazarız. Birleştirme alt alta yapılacak olsaydı '0' yazardık. Burada dikkat etmemiz gereken sütunları birleştirirken her iki verimizin de aynı satır sayısına sahip olması. Verileri alt alta birleştirecek olsaydık da verilerin aynı sütun sayısına sahip olması gerekecekti.

In [ ]:
y = dataset.iloc[:, 2].values

price kolonunu da y değişkenine atadık. Bu aşamadan sonra verilerimizi train ve test olarak bölmemiz gerekiyor. Bunun için de sci-kit learn kütüphanesini kullanıyoruz. Sci-kit learn hem veri ön işleme araçlarını hem de çok sayıda makine öğrenme algoritmalarını içermektedir.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, shuffle = True, random_state = 1)

sklearn kütüphanesinin model_selection sınıfı içerisindeki train_test_split fonksiyonu ile verilerimizi eğitim ve test olarak ayırdık. Bunun için fonksiyona parametre olarak X, y değerlerini verdik. Test için verimizin %25'ini ayıracağımızı bildirdik. Verilerin karışık seçilmesi için shuffle=true değerini seçtik. random_state parametresini için de 1 seçerek random veri seçiminin kodu her çalıştırdığımızda değişmemesini sağladık. Her seferinde eğitim ve test için seçilen veriler değişseydi kurduğumuz algoritmanın başarısını ölçmek daha güç olurdu.